In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import meteva.base as meb
import pandas as pd
import numpy as np
import datetime
import copy

随机生成包含多个层次，多个时刻，多个时效，多个成员的站点数据，用于测试示例

In [2]:
data = {"id":[54511,54522],"lon":[100,110],"lat":[30,40],"ob":[0,0],"grapes":[0,0],"ec":[0,0],}
df = pd.DataFrame(data)
sta = meb.sta_data(df)
meb.set_stadata_coords(sta,level = 1000,time = datetime.datetime(2019,12,31,8,0),dtime = 0)
sta1 = copy.deepcopy(sta)
meb.set_stadata_coords(sta1,level = 850)
sta = meb.combine_join(sta,sta1)
sta1 = copy.deepcopy(sta)
meb.set_stadata_coords(sta1,time = datetime.datetime(2020,1,1,20,0))
sta = meb.combine_join(sta,sta1)
sta1 = copy.deepcopy(sta)
meb.set_stadata_coords(sta1,dtime = 24)
sta_all = meb.combine_join(sta,sta1)
sta_all.iloc[:,-3:] = (np.random.rand(16,3)*100).astype(np.int16)
sta_all.iloc[10:13,-1] = meb.IV
print(sta_all)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  

# 通过综合字典选取数据  
**sele_by_dict(data,s):**   
通过包含多个维度选项的字典型参数，来确定所要选取的数据的范围，返回相应的数据样本。该函数和下文中sele_by_para函数是一一对应的，只是用于选择数据的参数形式以字典的形式统一成一个参数。
   
**参数说明：**  
 **data**: [站点数据](https://www.showdoc.cc/nmc?page_id=3744334022014027)        
 **s**: 字典型变量,字典的关键次为字符串，可选项和下文中sele_by_para的选取参数一一对应。具体请参见下文    
 **return**: [站点数据](https://www.showdoc.cc/nmc?page_id=3744334022014027)               

**调用示例**


In [3]:
sta = meb.sele_by_dict(sta_all,s = {"level":1000,"dtime":24,"member":["ob","ec"]}) 
print(sta) 

    level                time  dtime     id  lon  lat  ob      ec
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      61
12   1000 2020-01-01 20:00:00     24  54511  100   30  43  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97       2


In [4]:
#上述示例等价于
sta = meb.sele_by_para(sta_all,level=1000,dtime=24,member =["ob","ec"]) 
print(sta)

    level                time  dtime     id  lon  lat  ob      ec
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      61
12   1000 2020-01-01 20:00:00     24  54511  100   30  43  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97       2


#  通过参数提取部分数据   
**sele_by_para(data,member = None,level = None,time = None,time_range = None,year = None,month = None,day = None,dayofyear = None,hour = None,ob_time=None, ob_time_range=None, ob_year=None, ob_month=None, ob_day=None, ob_dayofyear=None, ob_hour=None,dtime = None,dtime_range = None,dday = None, dhour = None,lon = None,lat = None,id = None,gxy = None,gxyz = None,stadata = None,value = None,drop_IV = False,last = None,last_range = None)**   
从网格和站点数据中提取部分数据

|**参数**|   说明|  
|:-|:-----|  
|  **data**|[站点数据](https://www.showdoc.cc/nmc?page_id=3744334022014027)  |
|  **member**|        成员的名称，同时提取多个时采用列表形式  |
|  **level**|        层次的名称，同时提取多个时采用列表形式  |
|  **time**|         时间（以起报时刻为准），可以是datetime,datetime64或“2019010108”类似的字符串形式，同时提取多个时采用列表形式  |
|  **time_range**|    时间范围，列表形式，第一个元素为起始时间，第二个为结束时间，时间可以是datetime,datetime64或“2019010108”类似的字符串形式  |
|  **year**|         要提取的数据的年份（以起报时刻为准），同时提取多个时采用列表形式  |
|  **month**|        要提取的数据的月份（以起报时刻为准），同时提取多个时采用列表形式  |
|  **day**|          要提取的数据的日期（以起报时刻为准），可以是datetime,datetime64或“20190101”类似的字符串形式，同时提取多个时采用列表形式  |
|  **dayofyear**|     要提取的数据在一年中的排序（以起报时刻为准），整数形式，同时提取多个时采用列表形式  |
|  **hour**|         要提取的数据的小时数（以起报时刻为准），0-23的整数，同时提取多个时采用列表形式  |
|  **ob_time**|      时间（以观测时刻为准），可以是datetime,datetime64或“2019010108”类似的字符串形式，同时提取多个时采用列表形式  |
|  **ob_time_range**|  观测时间范围，列表形式，第一个元素为起始时间，第二个为结束时间，时间可以是datetime,datetime64或  “2019010108”类似的字符串形式|
|  **ob_year**|      要提取的数据的年份（以观测时刻为准），同时提取多个时采用列表形式 | 
|  **ob_month**|     要提取的数据的月份（以观测时刻为准），同时提取多个时采用列表形式  |
|  **ob_day**|       要提取的数据的日期（以观测时刻为准），可以是datetime,datetime64或“20190101”类似的字符串形式，同时提取多个时采用列表形式  |
|  **ob_dayofyear**|  要提取的数据在一年中的排序（以观测时刻为准），整数形式，同时提取多个时采用列表形式  |
|  **ob_hour**|      要提取的数据的小时数（以观测时刻为准），0-23的整数，同时提取多个时采用列表形式  |
|  **dtime**|        要提取的数据的时效，整数形式，同时提取多个是采用列表形式  ||
|  **dtime_range**|    时间范围，列表形式，第一个元素为起始时效，第二个为结束时效  |
|  **dday**|         要提取的数据的时效dtime整除以24的值，整数形式，同时提取多个时采用列表形式  |
|  **dhour**|        要提取的数据的时效dtime除24的余数，整数形式，同时提取多个时采用列表形式  |
|  **lon**|          要提取的数据的经度范围，列表形式，第一个元素为起始经度，第二个为结束经度  |
|  **lat**|         要提取的数据的纬度范围，列表形式，第一个元素为起始经度，第二个为结束经度  |
|  **id**|          要提取的数据的站号，同时提取多个是采用列表形式  |
|  **grid**|       [网格信息类](https://www.showdoc.cc/meteva?page_id=3975600815874861)变量，提取多维矩形网格范围内的数据，grid中size>1的维度的坐标范围会被用作选择的已经，size=1的维度会被忽略 | 
|  **gxy**|        [网格信息类](https://www.showdoc.cc/meteva?page_id=3975600815874861),采用其中经纬度范围提取水平矩形网格范围内的数据  |
|  **gxyz**|       [网格信息类](https://www.showdoc.cc/meteva?page_id=3975600815874861),采用其中经纬度和层次范围提取三维矩形网格范围内的数据   |
|  **stadata**|     [站点数据](https://www.showdoc.cc/nmc?page_id=3744334022014027), 对于stadata中level，time，dtime，id四个坐标中非缺省的部分，从data中提取和stadata坐标一致的站点数据  |
|  **value**|      提取所有数据列都在给定取值范围的数据，列表形式第一个元素为数据最低值，第二个为数据最高值  |
|  **drop_IV**|     该参数为True时，删除包含缺省值的行  |
|  **last**|       取出最后一列包含last的行，如何选择多个类型，last采用列表形式，并删除最后一列的数据  |
|  **last_range**|   包含起始值和结束值的列表，取出最后一列取值在该取值范围的数据，并删除最后一列的数据  |
|  **<font face="黑体" color=blue size=3>return**</font>|     [站点数据](https://www.showdoc.cc/nmc?page_id=3744334022014027)  |
  
**调用示例**

In [5]:
sta = meb.sele_by_para(sta_all,member = ["grapes"])  #提取单个成员的数据
print(sta)  

    level                time  dtime     id  lon  lat  grapes
0    1000 2019-12-31 08:00:00      0  54511  100   30      96
1    1000 2019-12-31 08:00:00      0  54522  110   40      78
2     850 2019-12-31 08:00:00      0  54511  100   30      31
3     850 2019-12-31 08:00:00      0  54522  110   40      13
4    1000 2020-01-01 20:00:00      0  54511  100   30      68
5    1000 2020-01-01 20:00:00      0  54522  110   40      30
6     850 2020-01-01 20:00:00      0  54511  100   30      46
7     850 2020-01-01 20:00:00      0  54522  110   40      70
8    1000 2019-12-31 08:00:00     24  54511  100   30      96
9    1000 2019-12-31 08:00:00     24  54522  110   40      22
10    850 2019-12-31 08:00:00     24  54511  100   30      47
11    850 2019-12-31 08:00:00     24  54522  110   40      11
12   1000 2020-01-01 20:00:00     24  54511  100   30      35
13   1000 2020-01-01 20:00:00     24  54522  110   40      58
14    850 2020-01-01 20:00:00     24  54511  100   30      95
15    85

In [6]:
sta = meb.sele_by_para(sta_all,member = ["ob","ec"])  #提取多个成员的数据
print(sta)  

    level                time  dtime     id  lon  lat  ob      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      41
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34       8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97       2
14    850 

In [7]:
sta = meb.sele_by_para(sta_all,level = 1000)  #提取指定层数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2


In [8]:
sta = meb.sele_by_para(sta_all,time = "2019123108")  #提取指定起报时刻数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999


In [9]:
sta = meb.sele_by_para(sta_all,year = 2020)  #提取指定起报年份数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


In [10]:
sta = meb.sele_by_para(sta_all,ob_year = 2020) #提取起报时刻+预报时效对应的观测时刻为指定年份的数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


In [11]:
sta = meb.sele_by_para(sta_all,dtime = 24) #提取指定时效数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


In [12]:
sta = meb.sele_by_para(sta_all,dtime_range = [0,24]) #提取指定时效范围的数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  

In [13]:
sta = meb.sele_by_para(sta_all,dday = 1) #提取时效>=1天 且 < 2天的数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


In [14]:
sta = meb.sele_by_para(sta_all,dhour = 0) #时效 % 1天的余数为0 的数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
11    850 2019-12-31 08:00:00     24  54522  110   40  22      11  999999
12   1000 2020-01-01 20:00:00     24  

In [15]:
sta = meb.sele_by_para(sta_all,id = 54511) #提取指定站点数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93


In [16]:
sta = meb.sele_by_para(sta_all,drop_IV=True) #删除包含缺省值的数据
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes  ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96  11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78  55
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31  86
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13  41
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68  10
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30   0
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46  26
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70   8
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96  12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22  61
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58   2
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95  93
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85  42


In [17]:
sta = meb.sele_by_para(sta_all,value=[20,80]) #删除包含缺省值的数据
print(sta)

   level                time  dtime     id  lon  lat  ob  grapes  ec
1   1000 2019-12-31 08:00:00      0  54522  110   40  71      78  55
9   1000 2019-12-31 08:00:00     24  54522  110   40  67      22  61


In [18]:
grid1 = meb.grid([100,105,1],[30,35,1],level_list=[925,700]) #grid1中有三个纬度size>1
sta = meb.sele_by_para(sta_all,grid = grid1)    #选择指定网格范围内的数据，
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93


In [19]:
sta = meb.sele_by_para(sta_all,gxy = grid1)   #仅采用grid1中水平方向的范围进行过滤
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
12   1000 2020-01-01 20:00:00     24  54511  100   30  43      35  999999
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93


In [20]:
sta = meb.sele_by_para(sta_all,gxyz = grid1)  #采用三维空间方向进行过滤
print(sta)

    level                time  dtime     id  lon  lat  ob  grapes      ec
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
14    850 2020-01-01 20:00:00     24  54511  100   30  75      95      93


In [21]:
loc =pd.DataFrame({"time":[datetime.datetime(2019,12,31,8),datetime.datetime(2020,1,1,20)],"id":[54511,54522]}) 
print(loc) #试验不同日期选择不同站点的方式

                 time     id
0 2019-12-31 08:00:00  54511
1 2020-01-01 20:00:00  54522


In [22]:
sta_loc = meb.sta_data(loc) 
print(sta_loc)  #要选择的站点序列，它在19年12月31日08时提取54511站，在1月1日20时提取54522站，level和dtime维度不做区分

   level                time  dtime     id  lon  lat  data0
0    NaN 2019-12-31 08:00:00    NaN  54511  NaN  NaN      0
1    NaN 2020-01-01 20:00:00    NaN  54522  NaN  NaN      0


In [23]:
sta = meb.sele_by_para(sta_all,stadata = sta_loc)  # 19年12月31日08时和54511和1月1日20时54522站的所有层次和时效数据被提取
print(sta)

   level                time  dtime     id  lon  lat  ob  grapes      ec
0   1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    850 2019-12-31 08:00:00      0  54511  100   30  25      31      86
2   1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0
3    850 2020-01-01 20:00:00      0  54522  110   40  34      70       8
4   1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12
5    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
6   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
7    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


如果数据选择的依据完全不基于level，time，dtime，id，lon，lat等时空坐标，比如以站点高度，下垫面类型作为选择依据时上述方法都无法涵盖，更有甚者如果我们选择涡度散度水汽条件这些条件来作为选择的依据，则这个判据是随所有的时空坐标的是变化的，此时就要用到lastR或lastL参数，方法如下。

In [24]:
sta_all["div"] = (np.random.randn(16) * 100).astype(np.int16)  
sta_all.iloc[0,-1] = 0
sta_all.iloc[1,-1] = 100
print(sta_all) # 首先我们需要在数据的最后一列增加一列，这一列是你选择的依据，它可以是散点，涡度，下垫面类型等等

    level                time  dtime     id  lon  lat  ob  grapes      ec  div
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11    0
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55  100
2     850 2019-12-31 08:00:00      0  54511  100   30  25      31      86 -144
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41   58
4    1000 2020-01-01 20:00:00      0  54511  100   30  30      68      10  -37
5    1000 2020-01-01 20:00:00      0  54522  110   40  24      30       0  -74
6     850 2020-01-01 20:00:00      0  54511  100   30  95      46      26   -9
7     850 2020-01-01 20:00:00      0  54522  110   40  34      70       8  -35
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      96      12 -117
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      22      61 -254
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999   95
11    850 2019-12-31 08:00:00     24  54522  110   4

In [25]:
sta = meb.sele_by_para(sta_all,last=[0,100]) 
print(sta) #选择最后一列取值在为0或100的数据，并删除最后一列，如果div代表下垫面类型，采用这种方式可以选择指定下垫面类型的数据

   level                time  dtime     id  lon  lat  ob  grapes  ec
0   1000 2019-12-31 08:00:00      0  54511  100   30  77      96  11
1   1000 2019-12-31 08:00:00      0  54522  110   40  71      78  55


In [26]:
sta = meb.sele_by_para(sta_all,last_range=[0,100]) 
print(sta) #选择最后一列取值在0到100范围的数据，并删除最后一列，如果div代表散度，则该方法相当于选择了所有散度在该范围的数据

    level                time  dtime     id  lon  lat  ob  grapes      ec
0    1000 2019-12-31 08:00:00      0  54511  100   30  77      96      11
1    1000 2019-12-31 08:00:00      0  54522  110   40  71      78      55
3     850 2019-12-31 08:00:00      0  54522  110   40  49      13      41
10    850 2019-12-31 08:00:00     24  54511  100   30  33      47  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97      58       2
15    850 2020-01-01 20:00:00     24  54522  110   40  22      85      42


In [27]:
sta = meb.sele_by_para(sta_all,level = 1000,dtime=24,member = ["ob","ec"]) 
print(sta)  #以上所有参数也可组合使用

    level                time  dtime     id  lon  lat  ob      ec
8    1000 2019-12-31 08:00:00     24  54511  100   30  13      12
9    1000 2019-12-31 08:00:00     24  54522  110   40  67      61
12   1000 2020-01-01 20:00:00     24  54511  100   30  43  999999
13   1000 2020-01-01 20:00:00     24  54522  110   40  97       2
